In [6]:
import pandas as pd
import numpy as np

In [8]:
pip install faker pandas

Note: you may need to restart the kernel to use updated packages.


In [56]:
from faker import Faker
import random

In [38]:
df_stateprobs = pd.read_csv('state_populations.csv')

In [49]:
total_population = df_stateprobs['Population'].sum()
df_stateprobs['Probability'] = df_stateprobs['Population'] / total_population
weights = df_stateprobs['Probability'].values

In [57]:
def generate_people(n: int, weights: list) -> pd.DataFrame:
    fake = Faker()
    data = []
    for _ in range(n):
        person = {
            'first_name': fake.first_name(),
            'last_name': fake.last_name(),
            'age': np.random.randint(18, 80),
            'us_state': random.choices(df_stateprobs['State'], weights=weights, k=1)[0],
            'random_sentence': fake.sentence()
        }
        data.append(person)
    return pd.DataFrame(data)

In [ ]:
n = 200
df_people = generate_people(n, weights)


  first_name last_name  age  us_state                     random_sentence
0     Ashley      Moss   47  Arkansas         Until number kid early cut.
1   Kristina  Fletcher   65  New York               Person day have blue.
2     Donald   Morales   74  Virginia        Radio son begin color image.
3   Mercedes    Phelps   45   Florida  Section remain respond put method.
4    Kenneth  Hamilton   63    Kansas    Person I color because personal.


In [60]:
df_people.to_csv('people.csv', index=False)

In [61]:
df_people = pd.read_csv('people.csv')

In [ ]:
populous_states = df_stateprobs.nlargest(n=10, columns=['Population'])

for i in populous_states['State']:
    state_people = df_people[df_people['us_state'] == i]
    
    # 4. Calculate the mean age for that specific state
    average_age = state_people['age'].mean()
    
    # Print the result (formatted)
    print(f"Average age in {i}: {average_age:.2f} years")

Average age in California: 50.68 years
Average age in Texas: 52.00 years
Average age in Florida: 54.00 years
Average age in New York: 49.38 years
Average age in Pennsylvania: 45.30 years
